#### Assi-7. Write a program to implement CNN using image dataset.

In [2]:
# prompt: extract faces.zip

import zipfile
import urllib.request

# Extract the contents of the zip file
with zipfile.ZipFile("Datasets\\faces.zip", 'r') as zip_ref:
    zip_ref.extractall("faces")

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import pandas as pd
from sklearn.model_selection import train_test_split
import os


# Load the CSV file
df = pd.read_csv('faces/train.csv')

# Define the directory containing the images
image_dir = 'faces/Train/'

# Split the data into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Define ImageDataGenerator for training and validation
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=image_dir,
    x_col='ID',
    y_col='Class',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=image_dir,
    x_col='ID',
    y_col='Class',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)



Found 15924 validated image filenames belonging to 3 classes.
Found 3982 validated image filenames belonging to 3 classes.


In [4]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Assuming 3 classes: YOUNG, MIDDLE, OLD
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


In [7]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=5
)


Epoch 1/5
199/497 [===========>..................] - ETA: 2:17 - loss: 0.8208 - accuracy: 0.6371

KeyboardInterrupt: 

In [6]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(val_generator)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')


125/125 [==============================] - 5s 42ms/step - loss: 0.6428 - accuracy: 0.7338
Validation Loss: 0.642762303352356
Validation Accuracy: 0.733802080154419


In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Function to preprocess the image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Rescale the image
    return img_array

# Function to make predictions
def predict_image(img_path, model):
    img_array = preprocess_image(img_path)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)
    return predicted_class

# Example usage
img_path = 'download.jpg'
predicted_class = predict_image(img_path, model)

# Map the predicted class index to the class label
class_labels = {0: 'YOUNG', 1: 'MIDDLE', 2: 'OLD'}
predicted_label = class_labels[predicted_class[0]]

print(f'Predicted class: {predicted_label}')


1/1 [==============================] - 0s 36ms/step
Predicted class: OLD
